In [1]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import time

import numpy as np
import dask
import distributed
import elfi
from distributed import Client

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use("ggplot")
%matplotlib inline


In [2]:
# Setup disributed
client = elfi.env.client()
dask.set_options(get=client.get)

In [3]:
def sleep(secs=10):
    time.sleep(secs)
    return secs

In [4]:
dsleep = dask.delayed(sleep, pure=True)
s10 = dsleep(10)

In [5]:
f = client.compute(s10)
time.sleep(7)

%time s10.compute()

CPU times: user 158 ms, sys: 17.5 ms, total: 176 ms
Wall time: 3.02 s


10

After it is finished, the scheduler forgets it, and traditional compute loses the benefit

In [6]:
%time s10.compute()

CPU times: user 408 ms, sys: 97.7 ms, total: 506 ms
Wall time: 10 s


10

We can see that the result exists, and using `client.compute` we can still benefit from it

In [7]:
%time client.compute(s10).result()

CPU times: user 4.65 ms, sys: 3.52 ms, total: 8.17 ms
Wall time: 6.77 ms


10

After we delete it, the benefit is lost

In [8]:
del f
time.sleep(1)
%time client.compute(s10).result()

CPU times: user 410 ms, sys: 69.6 ms, total: 479 ms
Wall time: 10 s


10

## Two levels

In [21]:
s5 = dsleep(5)
ss5 = dsleep(s5)
sss5 = dsleep(ss5)

No benefit for lower level delayed objects after they have been computed. Here the first timing benefits from the future, but the next one doesn't

In [22]:
f = client.compute(sss5)
time.sleep(3)

%time s5.compute()

time.sleep(6)

%time s5.compute()

CPU times: user 39.5 ms, sys: 19.4 ms, total: 58.8 ms
Wall time: 2.01 s
CPU times: user 129 ms, sys: 18.1 ms, total: 147 ms
Wall time: 5.02 s


5

In [11]:
#del f

In [12]:
%time ss5.compute()

CPU times: user 155 ms, sys: 33.3 ms, total: 189 ms
Wall time: 2.99 s


5

In [13]:
%time ss5.compute()

CPU times: user 347 ms, sys: 63.6 ms, total: 410 ms
Wall time: 10 s


5